In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas_profiling as pf #does eda with a few lines of codes
plt.style.use('seaborn-whitegrid')
sns.set_style('whitegrid')

%matplotlib inline


In [3]:
""""
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))  
""""

SyntaxError: invalid syntax (<ipython-input-3-dd45d365be4c>, line 1)

In [2]:
netflix=pd.read_csv(r'C:\Users\Olubukunola\Downloads\netflix_titles_nov_2019.csv.zip')

In [3]:
netflix.head(3)

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,type
0,81193313,Chocolate,NaN,"Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang ...",South Korea,"November 30, 2019",2019,TV-14,1 Season,"International TV Shows, Korean TV Shows, Roman...",Brought together by meaningful meals in the pa...,TV Show
1,81197050,Guatemala: Heart of the Mayan World,"Luis Ara, Ignacio Jaunsolo",Christian Morales,NaN,"November 30, 2019",2019,TV-G,67 min,"Documentaries, International Movies","From Sierra de las Minas to Esquipulas, explor...",Movie
2,81213894,The Zoya Factor,Abhishek Sharma,"Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, ...",India,"November 30, 2019",2019,TV-14,135 min,"Comedies, Dramas, International Movies",A goofy copywriter unwittingly convinces the I...,Movie


## Data Assessment

In [5]:
def data_ass(netflix):
    print('netflix movies and shows: ',df.shape[0])
    print('dataset variables: ',df.shape[1])
    print('-'*10)
    print('dateset columns: \n')
    print(df.columns)
    print('-'*10)
    print('data-type of each column: \n')
    print(df.dtypes)
    print('-'*10)
    print('missing rows in each column: \n')
    c=df.isnull().sum()
    print(c[c>0])
data_inv(df)

netflix movies and shows:  5837
dataset variables:  12
----------
dateset columns: 

Index(['show_id', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'type'],
      dtype='object')
----------
data-type of each column: 

show_id          int64
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
type            object
dtype: object
----------
missing rows in each column: 

director      1901
cast           556
country        427
date_added     642
rating          10
dtype: int64


In [4]:
netflix.shape

(5837, 12)

In [5]:
netflix.columns

Index(['show_id', 'title', 'director', 'cast', 'country', 'date_added',
       'release_year', 'rating', 'duration', 'listed_in', 'description',
       'type'],
      dtype='object')

In [6]:
netflix.describe()

,show_id,release_year
count,5.837000e+03,5837.000000
mean,7.730079e+07,2013.688539
std,9.479777e+06,8.419088
min,2.698800e+05,1925.000000
25%,8.004520e+07,2013.000000
50%,8.016353e+07,2016.000000
75%,8.024188e+07,2018.000000
max,8.122720e+07,2020.000000


In [7]:
netflix.info()

#checking the data type of each columnn

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5837 entries, 0 to 5836
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       5837 non-null   int64 
 1   title         5837 non-null   object
 2   director      3936 non-null   object
 3   cast          5281 non-null   object
 4   country       5410 non-null   object
 5   date_added    5195 non-null   object
 6   release_year  5837 non-null   int64 
 7   rating        5827 non-null   object
 8   duration      5837 non-null   object
 9   listed_in     5837 non-null   object
 10  description   5837 non-null   object
 11  type          5837 non-null   object
dtypes: int64(2), object(10)
memory usage: 547.3+ KB


In [9]:
netflix.isna().sum()
#checking for empty rows

show_id            0
title              0
director        1901
cast             556
country          427
date_added       642
release_year       0
rating            10
duration           0
listed_in          0
description        0
type               0
dtype: int64

Director, Cast, Country, Date_added, and rating have empty values in their columns

In [12]:
netflix.isnull().sum()

#this is also used to show empty values

show_id            0
title              0
director        1901
cast             556
country          427
date_added       642
release_year       0
rating            10
duration           0
listed_in          0
description        0
type               0
dtype: int64

We can show only the columns with empty values

In [14]:
empty_rows = netflix.isnull().sum()

empty_rows[empty_rows>0]

#this lists only the columns with empty rows

director      1901
cast           556
country        427
date_added     642
rating          10
dtype: int64

## Data Cleaning

* Drop id column
* Drop dublicate shows
* create a new column shows the number of cast in each row
* we have 10 missing rows in rating column, replace them by the mode
* for the missing rows in added_date column, replace them by January 1,{release_year}
* I think we can not replace missing rows in column country by other countries, but we can use genre to
* identify this country ex: replace missing rows by japan for Anime
* convert the date_added column from object type to datetime

In [16]:
duplicates =netflix.duplicated(['title','country','type','release_year'])
netflix[duplicates]

#to show duplicate rows

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,type
1134,80175351,Kakegurui,NaN,"Saori Hayami, Minami Tanaka, Tatsuya Tokutake,...",Japan,NaN,2019,TV-14,2 Seasons,"Anime Series, International TV Shows, TV Thril...",High roller Yumeko Jabami plans to clean house...,TV Show
1741,81072516,Sarkar,A.R. Murugadoss,"Vijay, Varalakshmi Sarathkumar, Keerthi Suresh...",India,"March 2, 2019",2018,TV-MA,162 min,"Action & Adventure, Dramas, International Movies",A ruthless businessman’s mission to expose ele...,Movie


In [19]:
netflix=netflix.drop_duplicates(['title','country','type','release_year'])

#to drop duplicate rows

In [20]:
netflix=netflix.drop('show_id',axis=1)

In [21]:
#df['cast'].isnull().values.any()

null = netflix['cast'].isnull()

print(null)
#to check if there are null values in my cast column

0       False
1       False
2       False
3       False
4       False
        ...  
5832    False
5833    False
5834    False
5835     True
5836    False
Name: cast, Length: 5835, dtype: bool


In [46]:
netflix['cast']=netflix['cast'].replace(np.nan,'Unknown') #replaces Nan with unknown

#thsi function counts the number of casts in the moves and returns the number of casts in a new column

def cast_counter(cast):
    if cast=='Unknown':
        return 0
    else:
        split=cast.split(', ')
        length=len(split)
        return length
    
netflix['number_of_cast']=netflix['cast'].apply(cast_counter)
netflix['cast']=netflix['cast'].replace('Unknown',np.nan)

In [47]:
netflix[['cast', 'number_of_cast']]

,cast,number_of_cast
0,"Ha Ji-won, Yoon Kye-sang, Jang Seung-jo, Kang ...",8
1,Christian Morales,1
2,"Sonam Kapoor, Dulquer Salmaan, Sanjay Kapoor, ...",8
3,"Mama Sane, Amadou Mbow, Ibrahima Traore, Nicol...",9
4,"Abigail Oliver, Andrea Libman, Briana Buckmast...",10
...,...,...
5832,"Nick Pawlow, Jordu Schell, Jay Kushwara, Micha...",10
5833,"Corey Feldman, Tony Todd, Tara Leigh, Erin Way...",6
5834,"Anders W. Berthelsen, Rebecka Hemse, Nikolaj L...",12
5835,NaN,0


In [49]:
df=df.reset_index()


In [16]:
df['rating']=df['rating'].fillna(df['rating'].mode()[0]

#this fills empty rows with the mode

In [17]:
df['date_added']=df['date_added'].fillna('January 1, {}'.format(str(df['release_year'].mode()[0])))

#this fills the empty rows with January and the release year

In [ ]:
for i,j in zip(df['country'].values,df.index):
    if i==np.nan:
        if ('Anime' in df.loc[j,'listed_in']) or ('anime' in df.loc[j,'listed_in']):
                df.loc[j,'country']='Japan'
        else:
            continue
    else:
        continue

In [ ]:
import re
months={
    'January':1,
    'February':2,
    'March':3,
    'April':4,
    'May':5,
    'June':6,
    'July':7,
    'August':8,
    'September':9,
    'October':10,
    'November':11,
    'December':12
}
date_lst=[]
for i in df['date_added'].values:
    str1=re.findall('([a-zA-Z]+)\s[0-9]+\,\s[0-9]+',i)
    str2=re.findall('[a-zA-Z]+\s([0-9]+)\,\s[0-9]+',i)
    str3=re.findall('[a-zA-Z]+\s[0-9]+\,\s([0-9]+)',i)
    date='{}-{}-{}'.format(str3[0],months[str1[0]],str2[0])
    date_lst.append(date)

In [8]:
def avg_mark(marks):
    sum_marks = sum(every_marks)
    total_marks = len(every_marks)
    avg_marks = sum_marks / total_marks
    return avg_marks

every_marks = [1, 4, 4, 67, 23, 43]
avg_mark(every_marks)

23.666666666666668

In [ ]:
df['date_added_cleaned']=date_lst

In [ ]:
df=df.drop('date_added',axis=1)

In [ ]:
df['date_added_cleaned']=df['date_added_cleaned'].astype('datetime64[ns]')

### Exploratory Data Analysis

now, it is time to answer some questions.

* Understand every category in rating column(Google it)
* Understanding what content is available in different countries.
* Is Netflix has increasingly focusing on TV rather than movies in recent years.
* The most observed rating categories in TV-shows and Movies
* Identifying similar content by matching text-based features
* How many content its release year differ from its year added

let's now google the categories and explore them

TV-MA:This program is specifically designed to be viewed by adults and therefore may be unsuitable for children under 17.
TV-14:This program contains some material that many parents would find unsuitable for children under 14 years of age.
TV-PG:This program contains material that parents may find unsuitable for younger children.
R:Under 17 requires accompanying parent or adult guardian,Parents are urged to learn more about the film before taking their young children with them.
PG-13:Some material may be inappropriate for children under 13. Parents are urged to be cautious. Some material may be inappropriate for pre-teenagers.
NR or UR:If a film has not been submitted for a rating or is an uncut version of a film that was submitted
PG:Some material may not be suitable for children,May contain some material parents might not like for their young children.
TV-Y7:This program is designed for children age 7 and above.
TV-G:This program is suitable for all ages.
TV-Y:Programs rated TV-Y are designed to be appropriate for children of all ages. The thematic elements portrayed in programs with this rating are specifically designed for a very young audience, including children ages 2-6.
TV-Y7-FV:is recommended for ages 7 and older, with the unique advisory that the program contains fantasy violence.
G:All ages admitted. Nothing that would offend parents for viewing by children.
NC-17:No One 17 and Under Admitted. Clearly adult. Children are not admitted.
here we discover that UR and NR is the same rating(unrated,Not rated)
Uncut/extended versions of films that are labeled "Unrated" also contain warnings saying that the uncut version of the film contains content that differs from the theatrical release and might not be suitable for minors.
so we have the fix this.

In [ ]:
plt.figure(figsize=(8,6))
df['rating'].value_counts(normalize=True).plot.bar()
plt.title('Distribution of rating categories')
plt.xlabel('rating')
plt.ylabel('relative frequency')
plt.show()

after expecting the distribution of rating ctegories, it is abvious that TV-MA is the mode
TV-14 comes in the second place and so on...

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(x='rating',hue='type',data=df)
plt.title('comparing frequency between type and rating')
plt.show()

We can say that Movies is the majority category in every rating category on Netflix, except(TV-Y,TV-Y7,TV-Y7-FV).

In [ ]:
df['country'].value_counts().sort_values(ascending=False)

In [ ]:
we have huge number of countries in country column so we will focus on (United States,India,United Kingdom,Japan,Canada,Spain)

In [ ]:
top_productive_countries=df[(df['country']=='United States')|(df['country']=='India')|(df['country']=='United Kingdom')|(df['country']=='Japan')|
                             (df['country']=='Canada')|(df['country']=='Spain')]
plt.figure(figsize=(10,8))
sns.countplot(x='country',hue='type',data=top_productive_countries)
plt.title('comparing between the types that the top countries produce')
plt.show()


UK and Japan produces TV-Shows more than Movies

In [ ]:
for i in top_productive_countries['country'].unique():
    print(i)
    print(top_productive_countries[top_productive_countries['country']==i]['rating'].value_counts(normalize=True)*100)
    print('-'*10)

now let's see if Netflix focuse on TV rather than movies in recent years(2017-2018-2019)

In [ ]:
df['year_added']=df['date_added_cleaned'].dt.year

In [ ]:
df['type'].value_counts(normalize=True)

relative freuency of movies still has the lead

In [ ]:
df.groupby('year_added')['type'].value_counts(normalize=True)*100

We can say that Netflix begin to focus on TV-Shows,but Movies still has the lead in evey year.

In [ ]:
dups=df.duplicated(['title'])
df[dups]['title']

In [ ]:
for i in df[dups]['title'].values:
    print(df[df['title']==i][['title','type','release_year','country']])
    print('-'*40)

In [ ]:
plt.figure(figsize=(10,8))
df['year_added'].value_counts().plot.bar()
plt.title('distribution of year-added')
plt.ylabel('relative frequency')
plt.xlabel('year_added')
plt.show()

2018 is remarkable year for netflix

In [ ]:
counts=0
for i,j in zip(df['release_year'].values,df['year_added'].values):
    if i!=j:
        counts+=1
print('number of contents that its release year differ from the year added to netflix are ',str(counts))